In [1]:
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [10]:
user_name = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')

In [12]:
cnx = mysql.connector.connect(user=user_name, password=password,host='localhost',database='BacterialGrowth')

In [13]:
# create cursor to read or write
cursor = cnx.cursor()

## Existing tables

In [14]:
pd.read_sql_query('show tables from BacterialGrowth',cnx)

,Tables_in_bacterialgrowth
0,Bacteria
1,BacteriaCommunity
2,CultivationConditions
3,Experiment
4,Precultivation
5,Reactor
6,TechnicalReplicates


In [15]:
pd.read_sql_query('describe {}'.format('Bacteria'), cnx)

,Field,Type,Null,Key,Default,Extra
0,bacteriaId,int,NO,UNI,None,
1,bacteriaGenus,varchar(100),YES,,,
2,bacteriaSpecies,varchar(100),NO,PRI,None,
3,bacteriaStrain,varchar(100),NO,PRI,None,


In [17]:
bacteria_tables = pd.read_sql_query('show tables from BacterialGrowth',cnx)
tables = bacteria_tables['Tables_in_bacterialgrowth']

for table_name in tables:
    output = pd.read_sql_query('describe {}'.format(table_name), cnx)
    print(table_name)
    print(output, '\n')

Bacteria
             Field          Type Null  Key Default Extra
0       bacteriaId           int   NO  UNI    None      
1    bacteriaGenus  varchar(100)  YES                   
2  bacteriaSpecies  varchar(100)   NO  PRI    None      
3   bacteriaStrain  varchar(100)   NO  PRI    None       

BacteriaCommunity
           Field Type Null  Key Default Extra
0     bacteriaId  int   NO  PRI    None      
1  cultivationId  int   NO  PRI    None       

CultivationConditions
                     Field          Type Null  Key Default Extra
0            cultivationId           int   NO  PRI    None      
1   cultivationDescription          text  YES         None      
2             experimentId           int  YES  MUL    None      
3         precultivationId           int  YES  MUL    None      
4    inoculumConcentration           int  YES         None      
5          inoculumVolumen           int  YES         None      
6              monoculture    tinyint(1)  YES            0      
7   

## Insert into tables

In [46]:
strains = ['L1-82', 'JCM 31262']

for st in strains:
    q = f""" INSERT IGNORE INTO Bacteria (bacteriaGenus, bacteriaSpecies, bacteriaStrain)
        VALUES ('Roseburia','Roseburia intestinalis','{st}')
        """
    cursor.execute(q)
    
q = f""" INSERT IGNORE INTO Bacteria (bacteriaGenus, bacteriaSpecies, bacteriaStrain)
        VALUES ('Bacteroides','Bacteroides thetaiotaomicron','VPI 5482')
        """
cursor.execute(q)

## Get information from the DB - SELECT queries

In [30]:
pd.read_sql('SELECT * FROM Bacteria WHERE bacteriaGenus = "Roseburia" LIMIT 10', cnx)

,bacteriaId,bacteriaGenus,bacteriaSpecies,bacteriaStrain
0,11,Roseburia,Roseburia intestinalis,L1-82
1,12,Roseburia,Roseburia intestinalis,JCM 31262


In [47]:
pd.read_sql('SELECT * FROM Bacteria LIMIT 10', cnx)

,bacteriaGenus,bacteriaSpecies,bacteriaStrain
0,Bacteroides,Bacteroides thetaiotaomicron,VPI 5482
1,Roseburia,Roseburia intestinalis,JCM 31262
2,Roseburia,Roseburia intestinalis,L1-82


## Create tables

In [39]:
query_create = f""" CREATE TABLE IF NOT EXISTS Bacteria (
        bacteriaGenus varchar(100), 
        bacteriaSpecies varchar(100),
        bacteriaStrain varchar(100),
        PRIMARY KEY (bacteriaSpecies, bacteriaStrain)
        )"""

#CREATE TABLE IF NOT EXISTS Bacteria (bacteriaGenus,bacteriaSpecies,bacteriaStrain,PRIMARY KEY (bacteriaSpecies, bacteriaStrain))

cursor.execute(query_create)